In [15]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model  import LogisticRegression, PassiveAggressiveClassifier, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score

from statsmodels.stats.contingency_tables import mcnemar
from scikit_posthocs import posthoc_nemenyi_friedman, posthoc_wilcoxon

from tqdm import tqdm

models = [DummyClassifier, LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis, LogisticRegression, DecisionTreeClassifier, KNeighborsClassifier, SVC, RandomForestClassifier, ExtraTreesClassifier, 
          GaussianNB, MLPClassifier, PassiveAggressiveClassifier]

RANDOM_SEED: int = 42

In [16]:
len(models)

12

# Dataset

First we have to download dataset from Kaggle: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

The dataset contains transactions made by credit cards in September 2013 by European cardholders. 
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

Due to the fact we won't perform feature engineering within this project we don't care about understanding of feature nature. For these reasons anomymised (PCA transformed) features are totally fine to test our classification pipeline for tunning and choosing the best classification model.

In [17]:
df = pd.read_csv('data/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [20]:
X, y = df.drop('Class', axis=1), df.Class

# Model evaluation

Lets define the common classification models from SK-Learn and split our dataset to train/val and test parts.

Train/val subset will be used for hyperparameter optmimization and test set for final evaluation.

Due to the fact we have unbalanced dataset, we want to apply stratification, so positive labels will be presented in both subsets in equal proportion.

Hyperparameter optimization is performed using cross-validation to have more robust evaluations.

In [18]:
print(f"Number of models: {len(models)}")
models

Number of models: 12


[sklearn.dummy.DummyClassifier,
 sklearn.discriminant_analysis.LinearDiscriminantAnalysis,
 sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis,
 sklearn.linear_model._logistic.LogisticRegression,
 sklearn.tree._classes.DecisionTreeClassifier,
 sklearn.neighbors._classification.KNeighborsClassifier,
 sklearn.svm._classes.SVC,
 sklearn.ensemble._forest.RandomForestClassifier,
 sklearn.ensemble._forest.ExtraTreesClassifier,
 sklearn.naive_bayes.GaussianNB,
 sklearn.neural_network._multilayer_perceptron.MLPClassifier,
 sklearn.linear_model._passive_aggressive.PassiveAggressiveClassifier]

In [21]:
# Stratification for balanced test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED, test_size=.2, stratify=y)

In [22]:
N_FOLDS: int = 5

In [23]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

In [24]:
parallelize = [RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier]

In [25]:
results = np.zeros((len(models), N_FOLDS))
cur_fold = 0
np.random.seed(RANDOM_SEED)

for train_index, test_index in skf.split(X_train, y_train):
    X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    for i, clf in tqdm(enumerate(models)):
        if clf not in parallelize:
            clf = clf().fit(X_train_cv, y_train_cv)
        else:
             clf = clf(n_jobs=-1).fit(X_train_cv, y_train_cv)
        score = f1_score(y_val, clf.predict(X_val))
        results[i, cur_fold] = score
    cur_fold += 1
    

0it [00:00, ?it/s]/Users/luzanikita/Software/anaconda3/lib/python3.8/site-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "
3it [00:08,  2.73s/it]/Users/luzanikita/Software/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
12it [08:16, 41.34s/it] 
0it [00:00, ?it/s]/Users/luzanikita/Software/anaconda3/lib/python3.8/site-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will 

# Statistical testing



In [26]:
from scipy.stats import friedmanchisquare

In [28]:
result = friedmanchisquare(*results.tolist())
result

FriedmanchisquareResult(statistic=53.32336182336185, pvalue=1.5678544859711209e-07)

*   H0: both models have the same performance
*   H1: performances of the two models are not equal

In [29]:
p = result.pvalue
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

Different distributions (reject H0)


Here we can see that at least 1 model is statistically significantly different that all others

In [ ]:
! pip3 install scikit-posthocs

In [107]:
results

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.8137931 , 0.83098592, 0.7972973 , 0.83870968, 0.79452055],
       [0.11774601, 0.11551724, 0.10500808, 0.124057  , 0.11083123],
       [0.66257669, 0.68874172, 0.725     , 0.7012987 , 0.67080745],
       [0.75324675, 0.78947368, 0.78431373, 0.70857143, 0.80519481],
       [0.09756098, 0.1627907 , 0.13953488, 0.13953488, 0.04938272],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.84285714, 0.84671533, 0.82758621, 0.84768212, 0.86111111],
       [0.84507042, 0.86330935, 0.85714286, 0.85333333, 0.85314685],
       [0.21029083, 0.25192802, 0.22171946, 0.25892857, 0.23614458],
       [0.55652174, 0.54545455, 0.24528302, 0.04878049, 0.23747681],
       [0.02409639, 0.        , 0.09638554, 0.        , 0.        ]])

In [56]:
results[1, :], results[4, :], results[-4, :],results[-5, :]

(array([0.8137931 , 0.83098592, 0.7972973 , 0.83870968, 0.79452055]),
 array([0.75324675, 0.78947368, 0.78431373, 0.70857143, 0.80519481]),
 array([0.84507042, 0.86330935, 0.85714286, 0.85333333, 0.85314685]),
 array([0.84285714, 0.84671533, 0.82758621, 0.84768212, 0.86111111]))

Let's compare best results

In [30]:
result = friedmanchisquare(results[1, :], results[4, :], results[-4, :],results[-5, :])
result

FriedmanchisquareResult(statistic=13.560000000000002, pvalue=0.0035695719978437583)

In [31]:
p = result.pvalue
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

Different distributions (reject H0)


In [32]:
next_mtr = np.array([results[1, :], results[4, :], results[-4, :],results[-5, :]])

In [33]:
posthoc_nemenyi_friedman(next_mtr.T)

,0,1,2,3
0,1.000000,0.597695,0.316064,0.203530
1,0.597695,1.000000,0.017331,0.007913
2,0.316064,0.017331,1.000000,0.900000
3,0.203530,0.007913,0.900000,1.000000


In [34]:
next_mtr

array([[0.8137931 , 0.83098592, 0.7972973 , 0.83870968, 0.79452055],
       [0.76129032, 0.77922078, 0.78431373, 0.70454545, 0.77124183],
       [0.82857143, 0.86330935, 0.85714286, 0.8590604 , 0.85314685],
       [0.84057971, 0.87142857, 0.82191781, 0.85526316, 0.86713287]])

We can see that samples -4 and -5 are rougly equal

In [35]:
models[-4], models[-5]

(sklearn.ensemble._forest.ExtraTreesClassifier,
 sklearn.ensemble._forest.RandomForestClassifier)

Let's compare them on a test set

In [36]:
clf1 = ExtraTreesClassifier(n_jobs=-1).fit(X_train, y_train)
clf2 = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train)

In [37]:
y_pred_clf1 = clf1.predict(X_test)
y_pred_clf2 = clf2.predict(X_test)

In [38]:
f1_score(y_test, y_pred_clf1)

0.8852459016393441

In [39]:
f1_score(y_test, y_pred_clf2)

0.8852459016393441

In [40]:
A = ((y_pred_clf1 == y_test) & (y_pred_clf2 == y_test)).sum()
B = ((y_pred_clf1 != y_test) & (y_pred_clf2 == y_test)).sum()
C = ((y_pred_clf1 == y_test) & (y_pred_clf2 != y_test)).sum()
D = ((y_pred_clf1 != y_test) & (y_pred_clf2 != y_test)).sum()

In [41]:
B + C

4

In [42]:
mcnemar([[A, B], [C, D]]).pvalue

1.0

In [43]:
contingency_table_df=pd.DataFrame(data={"nr_correct_clf1":["Yes/Yes","No/Yes"], "nr_incorrect_cl1":["Yes/No","No/No"]}, index=["nr_correct_clf2","nr_incorrect_clf2"])
contingency_table_df.iloc[0,0]=A
contingency_table_df.iloc[0,1]=B
contingency_table_df.iloc[1,0]=C
contingency_table_df.iloc[1,1]=D
contingency_table_df

,nr_correct_clf1,nr_incorrect_cl1
nr_correct_clf2,56939,2
nr_incorrect_clf2,2,19


Because models are not so different, let's choose the one with the best F1-score. 

In [86]:
results_df =  pd.DataFrame(results)

In [3]:
import pandas as pd
results_df = pd.read_csv('model_cmp.csv')

In [10]:
results_df.rename(columns={"Unnamed: 0": "Classifier"}, inplace=True)
results_df

,Classifier,fold_1,fold_2,fold_3,fold_4,fold_5
0,DummyClassifier,0.000000,0.000000,0.000000,0.000000,0.000000
1,LinearDiscriminantAnalysis,0.813793,0.830986,0.797297,0.838710,0.794521
2,QuadraticDiscriminantAnalysis,0.117746,0.115517,0.105008,0.124057,0.110831
3,LogisticRegression,0.662577,0.688742,0.725000,0.701299,0.670807
4,DecisionTreeClassifier,0.753247,0.789474,0.784314,0.708571,0.805195
5,KNeighborsClassifier,0.097561,0.162791,0.139535,0.139535,0.049383
6,SVC,0.000000,0.000000,0.000000,0.000000,0.000000
7,RandomForestClassifier,0.842857,0.846715,0.827586,0.847682,0.861111
8,ExtraTreesClassifier,0.845070,0.863309,0.857143,0.853333,0.853147
9,GaussianNB,0.210291,0.251928,0.221719,0.258929,0.236145


In [13]:
results_df.iloc[:, 1:] = results_df.iloc[:, 1:].apply(lambda x: round(x, 4))
results_df

,Classifier,fold_1,fold_2,fold_3,fold_4,fold_5
0,DummyClassifier,0.0000,0.0000,0.0000,0.0000,0.0000
1,LinearDiscriminantAnalysis,0.8138,0.8310,0.7973,0.8387,0.7945
2,QuadraticDiscriminantAnalysis,0.1177,0.1155,0.1050,0.1241,0.1108
3,LogisticRegression,0.6626,0.6887,0.7250,0.7013,0.6708
4,DecisionTreeClassifier,0.7532,0.7895,0.7843,0.7086,0.8052
5,KNeighborsClassifier,0.0976,0.1628,0.1395,0.1395,0.0494
6,SVC,0.0000,0.0000,0.0000,0.0000,0.0000
7,RandomForestClassifier,0.8429,0.8467,0.8276,0.8477,0.8611
8,ExtraTreesClassifier,0.8451,0.8633,0.8571,0.8533,0.8531
9,GaussianNB,0.2103,0.2519,0.2217,0.2589,0.2361


In [97]:
model_names = [x.__name__ for x in models]
results_df.index = model_names
results_df.columns = [f'fold_{i+1}' for i in range(N_FOLDS)]

In [99]:
results_df.to_csv('model_cmp.csv')

In [104]:
with open('data/X_train.pkl', 'wb') as f:
    pkl.dump(X_train, f)
with open('data/X_test.pkl', 'wb') as f:
    pkl.dump(X_test, f)
with open('data/y_train.pkl', 'wb') as f:
    pkl.dump(y_train, f)
with open('data/y_test.pkl', 'wb') as f:
    pkl.dump(y_test, f)